# Tools and Memory in LangChain

This notebook covers:

1. **Tools** - Allowing agents to 'act' in the real world
2. **MCP (Model Context Protocol)** - Connecting to external tool servers
3. **Memory** - Persisting state between agent invocations

---

## Setup

In [ ]:
%pip install -qU langchain langchain-openai langchain-community langgraph langchain-mcp-adapters

In [ ]:
import os
import getpass

def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")

_set_env("OPENAI_API_KEY")

---

# Part 1: Tools

Tools allow agents to interact with the real world - executing code, querying databases, calling APIs, and more.

LangChain supports many tool formats. The `@tool` decorator is the most common way to create tools.

## 1.1 Basic Tool Creation

The docstring and type hints are used by the LLM to determine when and how to call the tool:

In [ ]:
from typing import Literal
from langchain_core.tools import tool

@tool
def calculator(
    a: float, 
    b: float, 
    operation: Literal["add", "subtract", "multiply", "divide"]
) -> float:
    """Perform basic arithmetic operations on two numbers."""
    print(f"Calculator: {a} {operation} {b}")
    
    if operation == "add":
        return a + b
    elif operation == "subtract":
        return a - b
    elif operation == "multiply":
        return a * b
    elif operation == "divide":
        if b == 0:
            raise ValueError("Division by zero is not allowed.")
        return a / b
    else:
        raise ValueError(f"Invalid operation: {operation}")

In [ ]:
from langchain.agents import create_agent

agent = create_agent(
    model="openai:gpt-4o-mini",
    tools=[calculator],
    system_prompt="You are a helpful math assistant. Use the calculator tool for calculations.",
)

In [ ]:
# The agent will use the calculator tool
result = agent.invoke(
    {"messages": "What is 3.1415 * 2.7182?"}
)
print(result["messages"][-1].content)

In [ ]:
# For simple math, the agent might not use the tool
result = agent.invoke(
    {"messages": "What is 3 + 4?"}
)
print(result["messages"][-1].content)

## 1.2 Enhanced Tool Descriptions

Better descriptions help the LLM use tools more effectively. Use the `@tool` decorator with additional parameters:

In [ ]:
@tool(
    "math_calculator",
    parse_docstring=True,
    description=(
        "Perform arithmetic operations on two numbers. "
        "Use this for ANY mathematical calculation, even simple ones."
    ),
)
def enhanced_calculator(
    a: float, 
    b: float, 
    operation: Literal["add", "subtract", "multiply", "divide"]
) -> float:
    """Perform basic arithmetic operations on two numbers.

    Args:
        a: The first number (can be integer or decimal).
        b: The second number (can be integer or decimal).
        operation: The arithmetic operation to perform.
            - "add": Returns the sum of a and b.
            - "subtract": Returns a minus b.
            - "multiply": Returns the product of a and b.
            - "divide": Returns a divided by b (errors if b is zero).

    Returns:
        The numerical result of the operation.
    """
    print(f"Enhanced Calculator: {a} {operation} {b}")
    
    if operation == "add":
        return a + b
    elif operation == "subtract":
        return a - b
    elif operation == "multiply":
        return a * b
    elif operation == "divide":
        if b == 0:
            raise ValueError("Division by zero is not allowed.")
        return a / b

In [ ]:
agent = create_agent(
    model="openai:gpt-4o-mini",
    tools=[enhanced_calculator],
    system_prompt="You are a helpful math assistant.",
)

# Now even simple calculations should use the tool
result = agent.invoke({"messages": "What is 3 + 4?"})
print(result["messages"][-1].content)

## 1.3 Multiple Tools

Agents can use multiple tools and decide which ones to call:

In [ ]:
@tool
def get_current_time() -> str:
    """Get the current date and time."""
    from datetime import datetime
    return datetime.now().strftime("%Y-%m-%d %H:%M:%S")

@tool  
def get_random_number(min_val: int, max_val: int) -> int:
    """Generate a random integer between min_val and max_val (inclusive)."""
    import random
    return random.randint(min_val, max_val)

@tool
def reverse_string(text: str) -> str:
    """Reverse the characters in a string."""
    return text[::-1]

multi_tool_agent = create_agent(
    model="openai:gpt-4o-mini",
    tools=[enhanced_calculator, get_current_time, get_random_number, reverse_string],
    system_prompt="You are a helpful assistant with access to various tools.",
)

In [ ]:
result = multi_tool_agent.invoke(
    {"messages": "What time is it, and give me a random number between 1 and 100?"}
)
print(result["messages"][-1].content)

In [ ]:
result = multi_tool_agent.invoke(
    {"messages": "Reverse the word 'LangChain' and then tell me how many characters it has."}
)
print(result["messages"][-1].content)

### Try Your Own Tool

Create a tool below and test it:

In [ ]:
@tool
def your_custom_tool(input_param: str) -> str:
    """Describe what your tool does here.
    
    Args:
        input_param: Description of the input.
    
    Returns:
        Description of the output.
    """
    # Your implementation here
    return f"Processed: {input_param}"

# Test your tool
test_agent = create_agent(
    model="openai:gpt-4o-mini",
    tools=[your_custom_tool],
)

result = test_agent.invoke({"messages": "Use the custom tool with 'hello world'"})
print(result["messages"][-1].content)

---

# Part 2: MCP (Model Context Protocol)

MCP provides a standardized way to connect AI agents to external tools and data sources. Instead of defining tools in Python, you can connect to MCP servers that provide tools.

**Note:** This section requires `npx` (Node.js) to be installed for running MCP servers.

In [ ]:
from langchain_mcp_adapters.client import MultiServerMCPClient
import nest_asyncio

nest_asyncio.apply()

# Connect to the mcp-time server for timezone-aware operations
mcp_client = MultiServerMCPClient(
    {
        "time": {
            "transport": "stdio",
            "command": "npx",
            "args": ["-y", "@anthropic/mcp-server-time"],
        }
    },
)

# Load tools from the MCP server
mcp_tools = await mcp_client.get_tools()
print(f"Loaded {len(mcp_tools)} MCP tools: {[t.name for t in mcp_tools]}")

In [ ]:
from langchain.agents import create_agent

mcp_agent = create_agent(
    model="openai:gpt-4o-mini",
    tools=mcp_tools,
    system_prompt="You are a helpful assistant with access to time-related tools.",
)

In [ ]:
# Query the time in different timezones
result = await mcp_agent.ainvoke(
    {"messages": "What time is it in San Francisco and Tokyo right now?"}
)

for msg in result["messages"]:
    msg.pretty_print()

---

# Part 3: Memory - Persisting State

By default, agents don't remember previous conversations. Memory allows you to persist messages between invocations, enabling multi-turn conversations.

## 3.1 The Problem: No Memory

In [ ]:
from langchain_community.utilities import SQLDatabase
from dataclasses import dataclass
from langchain_core.tools import tool
from langgraph.runtime import get_runtime

db = SQLDatabase.from_uri("sqlite:///./assets-resources/Chinook.db")

@dataclass
class RuntimeContext:
    db: SQLDatabase

@tool
def execute_sql(query: str) -> str:
    """Execute a SQLite SELECT query and return results."""
    runtime = get_runtime(RuntimeContext)
    try:
        return runtime.context.db.run(query)
    except Exception as e:
        return f"Error: {e}"

SYSTEM_PROMPT = """You are a SQLite analyst. Use execute_sql for queries.
Limit to 5 rows unless asked otherwise."""

# Agent WITHOUT memory
agent_no_memory = create_agent(
    model="openai:gpt-4o-mini",
    tools=[execute_sql],
    system_prompt=SYSTEM_PROMPT,
    context_schema=RuntimeContext,
)

In [ ]:
# First question
for step in agent_no_memory.stream(
    {"messages": "This is Frank Harris. What was my last invoice total?"},
    context=RuntimeContext(db=db),
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

In [ ]:
# Follow-up question - agent doesn't remember who we are!
for step in agent_no_memory.stream(
    {"messages": "What were the track titles on that invoice?"},
    context=RuntimeContext(db=db),
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

## 3.2 The Solution: Add Memory

Use `InMemorySaver` as a checkpointer to persist conversation state:

In [ ]:
from langgraph.checkpoint.memory import InMemorySaver

# Agent WITH memory
agent_with_memory = create_agent(
    model="openai:gpt-4o-mini",
    tools=[execute_sql],
    system_prompt=SYSTEM_PROMPT,
    context_schema=RuntimeContext,
    checkpointer=InMemorySaver(),  # <-- This enables memory!
)

In [ ]:
# First question - use a thread_id to track the conversation
config = {"configurable": {"thread_id": "frank-session-1"}}

for step in agent_with_memory.stream(
    {"messages": "This is Frank Harris. What was my last invoice total?"},
    config,
    context=RuntimeContext(db=db),
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

In [ ]:
# Follow-up question - now the agent remembers!
for step in agent_with_memory.stream(
    {"messages": "What were the track titles on that invoice?"},
    config,  # Same thread_id
    context=RuntimeContext(db=db),
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

In [ ]:
# Another follow-up
for step in agent_with_memory.stream(
    {"messages": "What's the total price of those tracks?"},
    config,
    context=RuntimeContext(db=db),
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

## 3.3 Multiple Conversations with Thread IDs

Different `thread_id` values create separate conversation histories:

In [ ]:
# Start a NEW conversation with a different thread_id
new_config = {"configurable": {"thread_id": "another-user"}}

for step in agent_with_memory.stream(
    {"messages": "I'm a new user. What tables are in the database?"},
    new_config,
    context=RuntimeContext(db=db),
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

In [ ]:
# Go back to Frank's conversation - it still remembers
for step in agent_with_memory.stream(
    {"messages": "Remind me, what was my name again?"},
    config,  # Back to Frank's thread_id
    context=RuntimeContext(db=db),
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

### Try Your Own Conversation

Have a multi-turn conversation with the agent:

In [ ]:
my_config = {"configurable": {"thread_id": "my-conversation"}}

question = "What are the most popular genres in the database?"

for step in agent_with_memory.stream(
    {"messages": question},
    my_config,
    context=RuntimeContext(db=db),
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

---

## Summary

In this notebook, we covered:

1. **Tools** - Extending agent capabilities:
   - `@tool` decorator for creating tools
   - Type hints and docstrings guide the LLM
   - Enhanced descriptions with `parse_docstring=True`
   - Multiple tools in a single agent

2. **MCP** - External tool servers:
   - `MultiServerMCPClient` for connecting to MCP servers
   - Standard protocol for tool interoperability

3. **Memory** - Persisting state:
   - `InMemorySaver` as a checkpointer
   - `thread_id` for tracking conversations
   - Multi-turn conversations with context

---

**Next:** [Notebook 3: Advanced Patterns](./3.0-advanced-patterns.ipynb)